# Data Augmentation

In [7]:
# 필요한 라이브러리 설치
!pip install datasets tqdm albumentations opencv-python matplotlib pillow huggingface_hub

  Using cached albumentations-2.0.6-py3-none-any.whl.metadata (43 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached albumentations-2.0.6-py3-none-any.whl (332 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
Using cached pydantic-2.11.4-py3-none-any.whl (443 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)

   ---------------------------------------- 0/5 [opencv-python-headless]



ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\GitHubRepo\\NeuraSeg-HRNet-High-Resolution-MRI-Brain-Tumor-Detection-and-Segmentation\\.venv\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [ ]:
import os
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from datasets import load_dataset, Dataset, Features, Value, Sequence, Image, DatasetDict
from tqdm import tqdm
from PIL import Image as PILImage
from datetime import datetime
from huggingface_hub import login
import warnings
import traceback
import time
import shutil

warnings.filterwarnings('ignore')

# 허깅페이스 로그인 토큰 (실제 사용 시 수정 필요)
HF_TOKEN = ""  # 예시 토큰

# 결과 저장 경로
OUTPUT_PATH = "./augmented_data"
SPLITS = ['train', 'test', 'valid']

# 이미지와 어노테이션 저장 경로
OUTPUT_IMAGES_PATH = {split: os.path.join(OUTPUT_PATH, split) for split in SPLITS}
OUTPUT_ANNOTATIONS_FILE = {split: os.path.join(OUTPUT_PATH, f"{split}_annotations.json") for split in SPLITS}

# 주요 문제 해결 - 모든 데이터를 메모리에 저장한 후 한번에 처리하는 방식으로 변경
all_images = {split: {} for split in SPLITS}  # 원본 및 증강 이미지 저장
all_annotations = {split: [] for split in SPLITS}  # 어노테이션 저장

# 출력 디렉토리 초기화 및 생성
def setup_directories():
    """
    출력 디렉토리 초기화 및 생성
    """
    print("출력 디렉토리 설정 중...")
    try:
        # 기존 폴더가 있으면 삭제 (완전히 초기화)
        if os.path.exists(OUTPUT_PATH):
            print(f"기존 폴더 삭제 중: {OUTPUT_PATH}")
            shutil.rmtree(OUTPUT_PATH)
            time.sleep(1)  # 삭제 완료 대기
        
        # 새 폴더 생성
        os.makedirs(OUTPUT_PATH, exist_ok=True)
        for split in SPLITS:
            split_path = OUTPUT_IMAGES_PATH[split]
            os.makedirs(split_path, exist_ok=True)
            print(f"디렉토리 생성 완료: {split_path}")
            
        return True
    except Exception as e:
        print(f"디렉토리 설정 오류: {str(e)}")
        traceback.print_exc()
        return False

def load_dataset_from_huggingface():
    """
    허깅페이스에서 데이터셋 로드
    """
    print("허깅페이스에서 데이터셋 로드 중...")
    try:
        dataset = load_dataset("dwb2023/brain-tumor-image-dataset-semantic-segmentation")
        print("데이터셋 로드 완료!")
        return dataset
    except Exception as e:
        print(f"데이터셋 로드 오류: {str(e)}")
        traceback.print_exc()
        raise

def calculate_area_from_mask(mask):
    """
    마스크로부터 면적 계산
    """
    return float(np.sum(mask))

def create_mask_from_polygon(polygon, image_shape):
    """
    다각형 좌표로부터 마스크 생성
    """
    try:
        mask = np.zeros(image_shape[:2], dtype=np.uint8)
        
        if not polygon:
            return mask
        
        # 다각형 좌표 변환
        polygon_points = []
        for i in range(0, len(polygon), 2):
            polygon_points.append([int(polygon[i]), int(polygon[i+1])])
        
        polygon_array = np.array([polygon_points], dtype=np.int32)
        
        # 다각형 내부를 채우기
        cv2.fillPoly(mask, polygon_array, 1)
        
        return mask
    except Exception as e:
        print(f"마스크 생성 오류: {str(e)}")
        return np.zeros(image_shape[:2], dtype=np.uint8)

def calculate_bbox_from_mask(mask):
    """
    마스크로부터 바운딩 박스 계산
    """
    try:
        # 마스크에서 0이 아닌 값의 좌표 찾기
        rows = np.any(mask, axis=1)
        cols = np.any(mask, axis=0)
        
        if not np.any(rows) or not np.any(cols):
            return [0.0, 0.0, 0.0, 0.0]
        
        # 경계 좌표 찾기
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        
        # [x_min, y_min, x_max, y_max] 형식으로 반환
        return [float(x_min), float(y_min), float(x_max), float(y_max)]
    except Exception as e:
        print(f"바운딩 박스 계산 오류: {str(e)}")
        return [0.0, 0.0, 0.0, 0.0]

# 이미지 회전 및 뒤집기 함수
def rotate_image_90(image):
    """이미지를 90도 회전"""
    return np.rot90(image, k=1) if image is not None else None

def rotate_image_180(image):
    """이미지를 180도 회전"""
    return np.rot90(image, k=2) if image is not None else None

def rotate_image_270(image):
    """이미지를 270도 회전"""
    return np.rot90(image, k=3) if image is not None else None

def flip_image_horizontal(image):
    """이미지를 좌우 뒤집기"""
    return np.fliplr(image) if image is not None else None

def rotate_mask_90(mask, k=1):
    """마스크를 90도 회전"""
    return np.rot90(mask, k=k) if mask is not None else None

def flip_mask_horizontal(mask):
    """마스크를 좌우 뒤집기"""
    return np.fliplr(mask) if mask is not None else None

def visualize_augmentation_result(original_image, original_mask, augmented_image, augmented_mask, 
                                 title="Augmentation Result", save_path=None):
    """
    원본과 증강 결과 시각화
    """
    try:
        if original_image is None or augmented_image is None:
            print("시각화 오류: 이미지가 None입니다.")
            return
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))
        
        # 원본 이미지와 마스크
        ax1.imshow(original_image)
        ax1.set_title('원본 이미지')
        ax1.axis('off')
        
        # 원본 마스크
        ax2.imshow(original_mask, cmap='gray')
        ax2.set_title('원본 마스크')
        ax2.axis('off')
        
        # 증강 이미지와 마스크
        ax3.imshow(augmented_image)
        ax3.set_title('증강 이미지')
        ax3.axis('off')
        
        # 증강 마스크
        ax4.imshow(augmented_mask, cmap='gray')
        ax4.set_title('증강 마스크')
        ax4.axis('off')
        
        plt.suptitle(title)
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path)
            print(f"시각화 결과 저장: {save_path}")
        else:
            plt.show()
            
        plt.close()
    except Exception as e:
        print(f"시각화 오류: {str(e)}")
        traceback.print_exc()

def prepare_dataset(dataset):
    """
    모든 데이터셋 스플릿 준비
    """
    results = {}
    
    for split in SPLITS:
        print(f"\n===== {split.upper()} 데이터셋 준비 시작 =====")
        
        try:
            split_ds = dataset[split]
            print(f"{split} 데이터셋 크기: {len(split_ds)} 이미지")
            
            # 준비 작업
            for i in tqdm(range(len(split_ds)), desc=f"{split} 데이터 준비"):
                item = split_ds[i]
                
                # 필수 데이터 추출
                image_np = np.array(item['image'])
                
                # segmentation이 없거나 비어있으면 건너뛰기
                if 'segmentation' not in item or not item['segmentation']:
                    print(f"경고: {split} 항목 {i}에 segmentation이 없습니다.")
                    continue
                
                # ID 생성
                item_id = str(item.get('id', f"{split}_{i}"))
                
                # 원본 파일명 생성
                original_filename = f"original_{item_id}.png"
                
                # 마스크 생성 
                mask = create_mask_from_polygon(item['segmentation'][0], image_np.shape)
                
                # 메모리에 저장
                all_images[split][original_filename] = {
                    'image': image_np,
                    'mask': mask
                }
                
                # 메타데이터 생성
                meta = {
                    'id': item_id,
                    'file_name': original_filename,
                    'category_id': int(item.get('category_id', 0)),
                    'segmentation': item['segmentation'],
                    'height': int(image_np.shape[0]),
                    'width': int(image_np.shape[1])
                }
                
                # bbox 계산 (마스크 기반)
                meta['bbox'] = calculate_bbox_from_mask(mask)
                
                # area 계산 (마스크 기반)
                meta['area'] = calculate_area_from_mask(mask)
                
                # 추가 메타데이터 복사
                for field in ['iscrowd', 'license']:
                    if field in item:
                        meta[field] = item[field]
                
                # 어노테이션에 추가
                all_annotations[split].append(meta)
            
            # 결과 저장
            results[split] = {
                'original_count': len(all_annotations[split])
            }
            
            print(f"{split} 데이터셋 준비 완료: {results[split]['original_count']} 이미지")
            
        except Exception as e:
            print(f"{split} 데이터셋 준비 중 오류 발생: {str(e)}")
            traceback.print_exc()
            results[split] = {'original_count': 0}
    
    return results

def augment_dataset(visualization_path=None):
    """
    모든 데이터셋 스플릿 증강
    """
    # 증강 유형
    aug_types = {
        '90도_회전': {'image_func': rotate_image_90, 'mask_func': lambda m: rotate_mask_90(m, k=1)},
        '180도_회전': {'image_func': rotate_image_180, 'mask_func': lambda m: rotate_mask_90(m, k=2)},
        '270도_회전': {'image_func': rotate_image_270, 'mask_func': lambda m: rotate_mask_90(m, k=3)},
        '좌우_뒤집기': {'image_func': flip_image_horizontal, 'mask_func': flip_mask_horizontal}
    }
    
    results = {}
    
    # 시각화 디렉토리 생성
    if visualization_path:
        os.makedirs(visualization_path, exist_ok=True)
    
    for split in SPLITS:
        print(f"\n===== {split.upper()} 데이터셋 증강 시작 =====")
        
        try:
            original_annotations = all_annotations[split].copy()
            augmented_count = 0
            
            for idx, annotation in enumerate(tqdm(original_annotations, desc=f"{split} 이미지 증강 중")):
                original_filename = annotation['file_name']
                
                # 원본 이미지와 마스크 가져오기
                if original_filename not in all_images[split]:
                    print(f"경고: {original_filename}을 찾을 수 없습니다.")
                    continue
                    
                original_data = all_images[split][original_filename]
                original_image = original_data['image']
                original_mask = original_data['mask']
                
                # 각 증강 적용
                for aug_name, aug_funcs in aug_types.items():
                    # 증강 파일명
                    augmented_filename = f"aug_{annotation['id']}_{aug_name}.png"
                    
                    # 이미지와 마스크 증강
                    augmented_image = aug_funcs['image_func'](original_image)
                    augmented_mask = aug_funcs['mask_func'](original_mask)
                    
                    if augmented_image is None or augmented_mask is None:
                        print(f"경고: {augmented_filename} 증강 실패")
                        continue
                    
                    # 메모리에 저장
                    all_images[split][augmented_filename] = {
                        'image': augmented_image,
                        'mask': augmented_mask
                    }
                    
                    # bbox 및 area 계산
                    augmented_bbox = calculate_bbox_from_mask(augmented_mask)
                    augmented_area = calculate_area_from_mask(augmented_mask)
                    
                    # 어노테이션 생성
                    augmented_annotation = annotation.copy()
                    augmented_annotation['id'] = f"{annotation['id']}_aug_{aug_name}"
                    augmented_annotation['file_name'] = augmented_filename
                    augmented_annotation['bbox'] = augmented_bbox
                    augmented_annotation['area'] = augmented_area
                    augmented_annotation['height'] = augmented_image.shape[0]
                    augmented_annotation['width'] = augmented_image.shape[1]
                    augmented_annotation['augmentation_type'] = aug_name
                    
                    # segmentation 필드를 마스크로부터 재계산 (중요!)
                    # 실제로는 segmentation을 다각형으로 다시 변환해야 하지만,
                    # 여기서는 단순화를 위해 원본 segmentation 유지
                    
                    # 어노테이션 추가
                    all_annotations[split].append(augmented_annotation)
                    augmented_count += 1
                    
                    # 첫 5개 이미지에 대해 시각화
                    if visualization_path and idx < 5:
                        viz_filename = f"{split}_{annotation['id']}_{aug_name}.png"
                        viz_path = os.path.join(visualization_path, viz_filename)
                        visualize_augmentation_result(
                            original_image, original_mask, 
                            augmented_image, augmented_mask,
                            title=f"증강 결과 - {split} - {aug_name}",
                            save_path=viz_path
                        )
            
            # 결과 저장
            results[split] = {
                'original_count': len(original_annotations),
                'augmented_count': augmented_count,
                'total_count': len(all_annotations[split])
            }
            
            print(f"{split} 데이터셋 증강 완료: 원본 {results[split]['original_count']}개, "
                  f"증강 {results[split]['augmented_count']}개, 총 {results[split]['total_count']}개")
                  
        except Exception as e:
            print(f"{split} 데이터셋 증강 중 오류 발생: {str(e)}")
            traceback.print_exc()
            results[split] = {'original_count': 0, 'augmented_count': 0, 'total_count': 0}
    
    return results

def save_data_to_disk():
    """
    메모리에 있는 모든 데이터를 디스크에 저장
    """
    print("\n===== 모든 데이터를 디스크에 저장 중 =====")
    
    success_count = {split: 0 for split in SPLITS}
    error_count = {split: 0 for split in SPLITS}
    
    # 1. 이미지 저장
    for split in SPLITS:
        print(f"\n{split.upper()} 이미지 저장 중...")
        
        for filename, data in tqdm(all_images[split].items(), desc=f"{split} 이미지 저장"):
            try:
                # 이미지 경로
                image_path = os.path.join(OUTPUT_IMAGES_PATH[split], filename)
                
                # 이미지 저장 - PIL 사용 (더 안정적)
                image_pil = PILImage.fromarray(data['image'])
                image_pil.save(image_path, format='PNG')
                
                # 파일 존재 확인
                if os.path.exists(image_path):
                    success_count[split] += 1
                else:
                    print(f"경고: 이미지 파일이 생성되지 않음 - {image_path}")
                    error_count[split] += 1
                    
            except Exception as e:
                print(f"이미지 저장 오류 ({split}/{filename}): {str(e)}")
                error_count[split] += 1
    
    # 2. 어노테이션 저장
    for split in SPLITS:
        try:
            annotations_path = OUTPUT_ANNOTATIONS_FILE[split]
            with open(annotations_path, 'w', encoding='utf-8') as f:
                json.dump(all_annotations[split], f, indent=2, ensure_ascii=False)
                
            print(f"{split} 어노테이션 저장 완료: {annotations_path} ({len(all_annotations[split])}개)")
        except Exception as e:
            print(f"어노테이션 저장 오류 ({split}): {str(e)}")
    
    # 결과 요약
    print("\n===== 데이터 저장 결과 =====")
    for split in SPLITS:
        print(f"{split}: 성공 {success_count[split]}개, 오류 {error_count[split]}개")
    
    return all([error_count[split] == 0 for split in SPLITS])

def create_hf_dataset():
    """
    허깅페이스 데이터셋 생성 및 업로드
    """
    print("\n===== 허깅페이스 데이터셋 생성 중 =====")
    
    unified_dataset = {}
    
    for split in SPLITS:
        print(f"\n{split.upper()} 데이터셋 처리 중...")
        
        try:
            # 어노테이션 로드
            annotations = all_annotations[split]
            
            # 데이터 준비
            data = {
                "id": [],
                "file_name": [],
                "image": [],
                "category_id": [],
                "bbox": [],
                "area": [],
                "segmentation": []
            }
            
            # 추가 필드 확인
            extra_fields = set()
            for ann in annotations[:10]:
                for key in ann.keys():
                    if key not in data and key not in ["height", "width"]:
                        extra_fields.add(key)
                        data[key] = []
            
            # 데이터 채우기
            for idx, ann in enumerate(tqdm(annotations, desc=f"{split} 데이터셋 생성")):
                try:
                    # 필수 필드
                    data["id"].append(str(ann["id"]))
                    data["file_name"].append(ann["file_name"])
                    data["category_id"].append(int(ann["category_id"]))
                    data["bbox"].append(ann["bbox"])
                    data["area"].append(float(ann["area"]))
                    data["segmentation"].append(ann["segmentation"])
                    
                    # 추가 필드
                    for field in extra_fields:
                        if field in ann:
                            data[field].append(ann[field])
                        else:
                            data[field].append("")
                    
                    # 이미지 로드
                    image_path = os.path.join(OUTPUT_IMAGES_PATH[split], ann["file_name"])
                    if os.path.exists(image_path):
                        img = PILImage.open(image_path)
                        data["image"].append(img)
                    else:
                        print(f"경고: 이미지 파일을 찾을 수 없음 - {image_path}")
                        # 빈 이미지 사용
                        data["image"].append(PILImage.new('RGB', (640, 640), color='black'))
                        
                except Exception as e:
                    print(f"항목 처리 오류 ({split}/{idx}): {str(e)}")
            
            # 스키마 정의
            features = Features({
                "id": Value("string"),
                "file_name": Value("string"),
                "image": Image(),
                "category_id": Value("int64"),
                "bbox": Sequence(Value("float32")),
                "area": Value("float32"),
                "segmentation": Sequence(Sequence(Value("float32")))
            })
            
            # 추가 필드 스키마
            for field in extra_fields:
                features[field] = Value("string")
            
            # 데이터셋 생성
            unified_dataset[split] = Dataset.from_dict(data, features=features)
            print(f"{split} 데이터셋 생성 완료: {len(unified_dataset[split])} 샘플")
            
        except Exception as e:
            print(f"{split} 데이터셋 생성 오류: {str(e)}")
            traceback.print_exc()
    
    # 데이터셋 딕셔너리 생성
    try:
        if unified_dataset:
            dataset_dict = DatasetDict({
                split: unified_dataset[split] for split in unified_dataset.keys()
            })
            
            # 허깅페이스에 업로드
            print("\n허깅페이스에 데이터셋 업로드 중...")
            dataset_dict.push_to_hub(
                "espada105/augmented-brain-tumor-segmentation-v2",
                private=False
            )
            print("데이터셋 업로드 완료!")
            
            # 업로드된 데이터셋 요약
            print("\n===== 업로드된 데이터셋 요약 =====")
            for split in unified_dataset:
                print(f"{split}: {len(unified_dataset[split])} 샘플")
            
            total_samples = sum([len(unified_dataset[split]) for split in unified_dataset])
            print(f"총 샘플 수: {total_samples}")
            
            return True
        else:
            print("업로드할 데이터셋이 없습니다.")
            return False
    except Exception as e:
        print(f"데이터셋 업로드 오류: {str(e)}")
        traceback.print_exc()
        return False

def main():
    start_time = time.time()
    
    try:
        # 1. 디렉토리 설정
        if not setup_directories():
            print("오류: 디렉토리 설정 실패")
            return
        
        # 2. 허깅페이스 로그인
        login(token=HF_TOKEN)
        
        # 3. 데이터셋 로드
        dataset = load_dataset_from_huggingface()
        
        # 4. 시각화 디렉토리 생성
        viz_path = os.path.join(OUTPUT_PATH, "visualizations")
        os.makedirs(viz_path, exist_ok=True)
        
        # 5. 데이터셋 준비 (메모리에 로드)
        prep_results = prepare_dataset(dataset)
        
        # 6. 데이터셋 증강 (메모리 내에서)
        aug_results = augment_dataset(visualization_path=viz_path)
        
        # 7. 디스크에 모든 데이터 저장
        save_results = save_data_to_disk()
        
        if not save_results:
            print("경고: 일부 데이터가 제대로 저장되지 않았습니다.")
        
        # 8. 허깅페이스 데이터셋 생성 및 업로드
        create_hf_dataset()
        
        # 9. 처리 시간 계산
        end_time = time.time()
        processing_time = end_time - start_time
        print(f"\n총 처리 시간: {processing_time:.2f}초 ({processing_time/60:.2f}분)")
        
    except Exception as e:
        print(f"처리 중 오류 발생: {str(e)}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

출력 디렉토리 설정 중...
기존 폴더 삭제 중: ./augmented_data
디렉토리 생성 완료: ./augmented_data\train
디렉토리 생성 완료: ./augmented_data\test
디렉토리 생성 완료: ./augmented_data\valid
허깅페이스에서 데이터셋 로드 중...
데이터셋 로드 완료!

===== TRAIN 데이터셋 준비 시작 =====
train 데이터셋 크기: 1502 이미지


train 데이터 준비: 100%|██████████| 1502/1502 [00:05<00:00, 272.88it/s]


train 데이터셋 준비 완료: 1502 이미지

===== TEST 데이터셋 준비 시작 =====
test 데이터셋 크기: 215 이미지


test 데이터 준비: 100%|██████████| 215/215 [00:00<00:00, 247.16it/s]


test 데이터셋 준비 완료: 215 이미지

===== VALID 데이터셋 준비 시작 =====
valid 데이터셋 크기: 429 이미지


...

test 데이터셋 생성 완료: 1075 샘플

VALID 데이터셋 처리 중...


valid 데이터셋 생성: 100%|██████████| 2145/2145 [00:09<00:00, 223.88it/s]


valid 데이터셋 생성 완료: 2145 샘플

허깅페이스에 데이터셋 업로드 중...


Uploading the dataset shards: 100%|██████████| 1/1 [00:50<00:00, 50.40s/it]


데이터셋 업로드 완료!

===== 업로드된 데이터셋 요약 =====
train: 7510 샘플
test: 1075 샘플
valid: 2145 샘플
총 샘플 수: 10730

총 처리 시간: 729.66초 (12.16분)


# HuggingFace Upload

In [ ]:
!pip install datasets huggingface_hub pillow